In [1]:
import itertools
from collections import defaultdict
from tqdm import tqdm
import _pickle as cPickle
import os

que_split_path_dict = {

#     "minval":  "../../datasets/VQA/back-translate/org2_bt_v2_OpenEnded_mscoco_minval2014_questions.pkl",
    "val":  "../../datasets/VQA/back-translate/org2_bt_v2_OpenEnded_mscoco_val2014_questions.pkl",
    "train": "../../datasets/VQA/back-translate/org2_bt_v2_OpenEnded_mscoco_train2014_questions.pkl",
}

answer_paths = [
#     "../../datasets/VQA/back-translate/org_bt_minval_target.pkl",
    "../../datasets/VQA/back-translate/org2_bt_val_target.pkl",
    "../../datasets/VQA/back-translate/org2_bt_train_target.pkl",
    ]


ans2label_path = os.path.join("../../datasets/VQA/", "cache", "trainval_ans2label.pkl")
label2ans_path = os.path.join("../../datasets/VQA/", "cache", "trainval_label2ans.pkl")

ans2label = cPickle.load(open(ans2label_path, "rb"))
label2ans = cPickle.load(open(label2ans_path, "rb"))
answer_dict = {}
# use later
answers_data = []

for path in (answer_paths):
    answers = cPickle.load(open(path, "rb"))
    answers_data.append(answers)
    answers = list(itertools.chain.from_iterable(answers))
    for ans in answers:
        answer_dict[ans["question_id"]] = ans["labels"]

print("Read Answers")

negs_path = "../../datasets/VQA/back-translate/train_val_question_negs.pkl"
negs_data = cPickle.load(open(negs_path, "rb"))
negs_dict = {}
for qid, sim_scores, sim_qids in zip(negs_data["qids"], negs_data["sim_scores"], negs_data["sim_qids"]):
    negs_dict[qid] = (sim_scores, sim_qids)

print("Read Matrix")

# create dicts
image_dict = defaultdict(list)
questions_rephrasings = defaultdict(list)
question_dict = {}

# use later
questions_data = []
qids_dict = {}
for split, que_path in que_split_path_dict.items():
    data = cPickle.load(open(que_path, "rb"))
    questions_data.append(data)
    questions_list = data["questions"]
    _dict = {}
    
    # add "rephrasing_of" key
    for _questions in questions_list:
        # only keep the min-qid in same-image ids
        min_qid = min([x['question_id'] for x in _questions])
        assert len(set([x['image_id'] for x in _questions])) == 1
        image_dict[_questions[0]["image_id"]].append(min_qid)
        for _que in _questions:
            question_dict[_que["question_id"]] = _que["question"]
            _dict[_que["question_id"]] = None
    qids_dict[split] = _dict

print("Read Questions")



Read Answers
Read Matrix
Read Questions


In [2]:
save_negs_dict_train = {}
save_negs_dict_val = {}
save_negs_dict_trainval = {}

def filter_negatives(sample):
    # filter same-image questions
    save_dict = {}
    same_image_ids = image_dict[sample["image_id"]]
    fil_same_image_ids = []
    ref_answers = answer_dict[sample["question_id"]]
    for qid in same_image_ids:
        if qid == sample["question_id"]:
            continue
        cand_answers = answer_dict[qid]
        if len(set(ref_answers).intersection(set(cand_answers))) == 0:
            fil_same_image_ids.append(qid)
    save_dict["same_image_questions_neg"] = fil_same_image_ids

    # filter top-k questions
    if sample["question_id"] not in negs_dict:
        return True

    top_k_sim_scores, top_k_questions = negs_dict[sample["question_id"]]
    fil_top_k_questions = []
    fil_top_k_questions_train = []
    fil_top_k_questions_val = []
    
    for qid in top_k_questions:
        cand_answers = answer_dict[qid]
        if len(set(ref_answers).intersection(set(cand_answers))) == 0:
            fil_top_k_questions.append(qid)
            if qid in qids_dict["train"]:
                fil_top_k_questions_train.append(qid)
            elif qid in qids_dict["val"]:
                fil_top_k_questions_val.append(qid)
            else:
                import pdb
                pdb.set_trace()
    
    save_dict["top_k_questions_neg"] = fil_top_k_questions
    save_dict["top_k_questions_neg_train"] = fil_top_k_questions_train
    save_dict["top_k_questions_neg_val"] = fil_top_k_questions_val

    if sample["question_id"] in qids_dict["train"]:
        del save_dict["top_k_questions_neg_val"]
        save_negs_dict_trainval[sample["question_id"]] = save_dict
        save_negs_dict_train[sample["question_id"]] = save_dict
    elif sample["question_id"] in qids_dict["val"]:
        del save_dict["top_k_questions_neg_train"]
        save_negs_dict_trainval[sample["question_id"]] = save_dict
        save_negs_dict_val[sample["question_id"]] = save_dict
    else:
        import pdb
        pdb.set_trace() 
    return False

In [3]:
for que_data, ans_data, que_path, ans_path in zip(questions_data, answers_data, que_split_path_dict.values(), answer_paths):
    data = que_data
    answers = ans_data

    questions_list = data["questions"]

    # add "rephrasing_of" key
    for _questions in questions_list:
        rep_id = min([s['question_id'] for s in _questions])
        for _que in _questions:
            _que["rephrasing_of"] = rep_id

    assert len(questions_list) == len(answers)

    # remove questions w/o negatives
    for idx in tqdm(range(len(questions_list)), total=len(questions_list)):
        _updated_ques = []
        _updated_answers = []
        _questions = questions_list[idx]
        _answers = answers[idx]

        for _que, _ans in zip(_questions, _answers):
            delete = filter_negatives(_que)
            if not delete:
                _updated_ques.append(_que)
                _updated_answers.append(_ans)
        questions_list[idx] = _updated_ques
        answers[idx] = _updated_answers

    cPickle.dump(data, open(que_path, "wb"), protocol=2)
    cPickle.dump(answers, open(ans_path, "wb"), protocol=2)
    print(f"Dumped: {que_path}")
    print(f"Dumped: {ans_path}")
    


100%|██████████| 214354/214354 [11:12<00:00, 318.77it/s]


Dumped: ../../datasets/VQA/back-translate/org2_bt_v2_OpenEnded_mscoco_val2014_questions.pkl
Dumped: ../../datasets/VQA/back-translate/org2_bt_val_target.pkl


100%|██████████| 443757/443757 [23:57<00:00, 308.77it/s]  


Dumped: ../../datasets/VQA/back-translate/org2_bt_v2_OpenEnded_mscoco_train2014_questions.pkl
Dumped: ../../datasets/VQA/back-translate/org2_bt_train_target.pkl


In [41]:
save_negs_paths = [
"../../datasets/VQA/back-translate/fil_val_question_negs.pkl",
"../../datasets/VQA/back-translate/fil_train_question_negs.pkl",
"../../datasets/VQA/back-translate/fil_trainval_question_negs.pkl",
]

# _dicts = [save_negs_dict_val,
# save_negs_dict_train,
# save_negs_dict_trainval]

# for path, split, _d in zip(save_negs_paths, ["val", "train", "trainval"], _dicts):
#     cPickle.dump(_d, open(path, "wb"), protocol=2)
#     print(f"Dumped: {path}")


In [ ]:
values = list(save_negs_dict_val.values())
lens = [len(x["top_k_questions_neg"]) for x in values]
max(lens)

In [37]:
import numpy as np

question_ids = list(save_negs_dict_val.keys())
question_values = list(save_negs_dict_val.values())
question_negs = np.zeros((len(question_ids), 300), dtype=np.int32) + -1
same_image_questions_neg = np.zeros((len(question_ids), 300), dtype=np.int32) + -1

for idx in tqdm(range(len(question_negs))):
    neg_ids = question_values[idx]["top_k_questions_neg_val"]
    same_image_ids = question_values[idx]["same_image_questions_neg"]
    question_negs[idx][:len(neg_ids)] = neg_ids
    try:
        same_image_questions_neg[idx][:len(same_image_ids)] = same_image_ids
    except:
        import pdb
        pdb.set_trace()




  0%|          | 0/1024112 [00:00<?, ?it/s]


  1%|          | 6950/1024112 [00:00<00:14, 69496.42it/s]


  1%|▏         | 13977/1024112 [00:00<00:14, 69726.47it/s]


  2%|▏         | 21037/1024112 [00:00<00:14, 69986.17it/s]


  3%|▎         | 28283/1024112 [00:00<00:14, 70353.91it/s]


  3%|▎         | 35067/1024112 [00:00<00:14, 69579.49it/s]


  4%|▍         | 42170/1024112 [00:00<00:14, 70005.20it/s]


  5%|▍         | 49499/1024112 [00:00<00:13, 70956.96it/s]


  6%|▌         | 56938/1024112 [00:00<00:13, 71951.16it/s]


  6%|▋         | 64331/1024112 [00:00<00:13, 72533.32it/s]


  7%|▋         | 71500/1024112 [00:01<00:13, 72278.21it/s]


  8%|▊         | 78536/1024112 [00:01<00:13, 70157.30it/s]


  8%|▊         | 85762/1024112 [00:01<00:13, 70774.53it/s]


  9%|▉         | 92757/1024112 [00:01<00:13, 70329.39it/s]


 10%|▉         | 100100/1024112 [00:01<00:12, 71231.60it/s]


 10%|█         | 107186/1024112 [00:01<00:12, 70869.12it/s]


 11%|█         | 114285/1024112 [00

 81%|████████  | 824535/1024112 [00:11<00:02, 69951.88it/s]


 81%|████████  | 831714/1024112 [00:12<00:02, 70487.93it/s]


 82%|████████▏ | 838846/1024112 [00:12<00:02, 70735.02it/s]


 83%|████████▎ | 845940/1024112 [00:12<00:02, 70337.52it/s]


 83%|████████▎ | 853117/1024112 [00:12<00:02, 70759.69it/s]


 84%|████████▍ | 860204/1024112 [00:12<00:02, 67677.84it/s]


 85%|████████▍ | 867008/1024112 [00:12<00:02, 65148.88it/s]


 85%|████████▌ | 873987/1024112 [00:12<00:02, 66473.79it/s]


 86%|████████▌ | 880673/1024112 [00:12<00:02, 65240.64it/s]


 87%|████████▋ | 887626/1024112 [00:12<00:02, 66467.44it/s]


 87%|████████▋ | 894301/1024112 [00:12<00:01, 65923.01it/s]


 88%|████████▊ | 900914/1024112 [00:13<00:01, 65258.02it/s]


 89%|████████▊ | 908209/1024112 [00:13<00:01, 67388.94it/s]


 89%|████████▉ | 915587/1024112 [00:13<00:01, 69186.27it/s]


 90%|█████████ | 922538/1024112 [00:13<00:01, 69246.46it/s]


 91%|█████████ | 929797/1024112 [00:13<00:01, 70213.59it/s]


 92%|███

In [43]:
cPickle.dump({"qids": question_ids, 
                    "question_negs": question_negs, 
                    "same_image_questions_neg": same_image_questions_neg}, 
             open(save_negs_paths[0], "wb"))


In [44]:
import numpy as np

question_ids = list(save_negs_dict_train.keys())
question_values = list(save_negs_dict_train.values())
question_negs = np.zeros((len(question_ids), 300), dtype=np.int32) + -1
same_image_questions_neg = np.zeros((len(question_ids), 300), dtype=np.int32) + -1

for idx in tqdm(range(len(question_negs))):
    neg_ids = question_values[idx]["top_k_questions_neg_train"]
    same_image_ids = question_values[idx]["same_image_questions_neg"]
    question_negs[idx][:len(neg_ids)] = neg_ids
    try:
        same_image_questions_neg[idx][:len(same_image_ids)] = same_image_ids
    except:
        import pdb
        pdb.set_trace()




  0%|          | 0/2119034 [00:00<?, ?it/s]


  0%|          | 4769/2119034 [00:00<00:44, 47679.93it/s]


  0%|          | 9210/2119034 [00:00<00:45, 46647.81it/s]


  1%|          | 13745/2119034 [00:00<00:45, 46249.16it/s]


  1%|          | 18313/2119034 [00:00<00:45, 46075.32it/s]


  1%|          | 22112/2119034 [00:00<00:48, 43303.10it/s]


  1%|▏         | 26620/2119034 [00:00<00:47, 43821.24it/s]


  1%|▏         | 30469/2119034 [00:00<00:51, 40633.27it/s]


  2%|▏         | 35029/2119034 [00:00<00:49, 42004.95it/s]


  2%|▏         | 39640/2119034 [00:00<00:48, 43157.33it/s]


  2%|▏         | 43940/2119034 [00:01<00:48, 43107.80it/s]


  2%|▏         | 48150/2119034 [00:01<00:49, 41878.32it/s]


  2%|▏         | 52275/2119034 [00:01<00:53, 38808.44it/s]


  3%|▎         | 56502/2119034 [00:01<00:51, 39783.18it/s]


  3%|▎         | 61024/2119034 [00:01<00:49, 41271.10it/s]


  3%|▎         | 65857/2119034 [00:01<00:47, 43161.53it/s]


  3%|▎         | 70521/2119034 [00:01<

 23%|██▎       | 478243/2119034 [00:12<00:42, 38276.93it/s]


 23%|██▎       | 482596/2119034 [00:12<00:41, 39712.06it/s]


 23%|██▎       | 486795/2119034 [00:12<00:40, 40365.88it/s]


 23%|██▎       | 491293/2119034 [00:12<00:39, 41645.37it/s]


 23%|██▎       | 495671/2119034 [00:12<00:38, 42261.37it/s]


 24%|██▎       | 499923/2119034 [00:12<00:39, 40634.39it/s]


 24%|██▍       | 504066/2119034 [00:12<00:39, 40865.59it/s]


 24%|██▍       | 508175/2119034 [00:12<00:39, 40649.52it/s]


 24%|██▍       | 512256/2119034 [00:12<00:39, 40244.11it/s]


 24%|██▍       | 516293/2119034 [00:13<00:42, 38127.31it/s]


 25%|██▍       | 520138/2119034 [00:13<00:42, 37389.30it/s]


 25%|██▍       | 524357/2119034 [00:13<00:41, 38709.69it/s]


 25%|██▍       | 528258/2119034 [00:13<00:46, 34306.15it/s]


 25%|██▌       | 532603/2119034 [00:13<00:43, 36617.83it/s]


 25%|██▌       | 536388/2119034 [00:13<00:43, 36268.31it/s]


 26%|██▌       | 540776/2119034 [00:13<00:41, 38258.06it/s]


 26%|██▌

 44%|████▍     | 936586/2119034 [00:24<00:34, 33857.98it/s]


 44%|████▍     | 940197/2119034 [00:24<00:34, 34502.46it/s]


 45%|████▍     | 943698/2119034 [00:24<00:34, 33881.38it/s]


 45%|████▍     | 947124/2119034 [00:24<00:35, 33408.82it/s]


 45%|████▍     | 950493/2119034 [00:24<00:35, 33199.94it/s]


 45%|████▌     | 954560/2119034 [00:24<00:33, 35135.70it/s]


 45%|████▌     | 958339/2119034 [00:24<00:32, 35891.69it/s]


 45%|████▌     | 962462/2119034 [00:24<00:30, 37339.93it/s]


 46%|████▌     | 966236/2119034 [00:25<00:30, 37290.69it/s]


 46%|████▌     | 970514/2119034 [00:25<00:29, 38781.47it/s]


 46%|████▌     | 974427/2119034 [00:25<00:30, 37695.99it/s]


 46%|████▌     | 978228/2119034 [00:25<00:30, 37032.29it/s]


 46%|████▋     | 982425/2119034 [00:25<00:29, 38385.98it/s]


 47%|████▋     | 986407/2119034 [00:25<00:29, 38803.39it/s]


 47%|████▋     | 990703/2119034 [00:25<00:28, 39961.97it/s]


 47%|████▋     | 994724/2119034 [00:25<00:29, 38537.02it/s]


 47%|███

 66%|██████▌   | 1390797/2119034 [00:36<00:17, 41343.21it/s]


 66%|██████▌   | 1394954/2119034 [00:36<00:18, 39595.82it/s]


 66%|██████▌   | 1399064/2119034 [00:36<00:17, 40034.78it/s]


 66%|██████▌   | 1403091/2119034 [00:36<00:19, 35914.34it/s]


 66%|██████▋   | 1406897/2119034 [00:36<00:19, 36531.55it/s]


 67%|██████▋   | 1411214/2119034 [00:36<00:18, 38296.68it/s]


 67%|██████▋   | 1415301/2119034 [00:36<00:18, 39032.99it/s]


 67%|██████▋   | 1419262/2119034 [00:36<00:17, 39201.81it/s]


 67%|██████▋   | 1423283/2119034 [00:37<00:17, 39498.54it/s]


 67%|██████▋   | 1427260/2119034 [00:37<00:17, 39520.47it/s]


 68%|██████▊   | 1431446/2119034 [00:37<00:17, 40194.38it/s]


 68%|██████▊   | 1435482/2119034 [00:37<00:17, 39662.32it/s]


 68%|██████▊   | 1439603/2119034 [00:37<00:16, 40112.28it/s]


 68%|██████▊   | 1443625/2119034 [00:37<00:17, 38565.45it/s]


 68%|██████▊   | 1447502/2119034 [00:37<00:19, 35232.53it/s]


 69%|██████▊   | 1451737/2119034 [00:37<00:17, 37102.17

 86%|████████▌ | 1813787/2119034 [00:48<00:08, 34468.54it/s]


 86%|████████▌ | 1817387/2119034 [00:48<00:08, 34910.91it/s]


 86%|████████▌ | 1820915/2119034 [00:48<00:09, 32592.90it/s]


 86%|████████▌ | 1824501/2119034 [00:48<00:08, 33508.13it/s]


 86%|████████▋ | 1828520/2119034 [00:48<00:08, 35266.76it/s]


 86%|████████▋ | 1832125/2119034 [00:48<00:08, 35496.55it/s]


 87%|████████▋ | 1835715/2119034 [00:48<00:08, 34227.06it/s]


 87%|████████▋ | 1839503/2119034 [00:49<00:07, 35246.46it/s]


 87%|████████▋ | 1843177/2119034 [00:49<00:07, 35681.16it/s]


 87%|████████▋ | 1847420/2119034 [00:49<00:07, 37465.95it/s]


 87%|████████▋ | 1851711/2119034 [00:49<00:06, 38947.18it/s]


 88%|████████▊ | 1855780/2119034 [00:49<00:06, 39452.81it/s]


 88%|████████▊ | 1860087/2119034 [00:49<00:06, 40472.02it/s]


 88%|████████▊ | 1864164/2119034 [00:49<00:06, 40015.34it/s]


 88%|████████▊ | 1868402/2119034 [00:49<00:06, 40693.89it/s]


 88%|████████▊ | 1872490/2119034 [00:49<00:06, 40721.88

In [45]:
cPickle.dump({"qids": question_ids, 
                    "question_negs": question_negs, 
                    "same_image_questions_neg": same_image_questions_neg}, 
             open(save_negs_paths[1], "wb"))
print(save_negs_paths[1])

../../datasets/VQA/back-translate/fil_train_question_negs.pkl


In [ ]:
import numpy as np

question_ids = list(save_negs_dict_trainval.keys())
question_values = list(save_negs_dict_trainval.values())
question_negs = np.zeros((len(question_ids), 300), dtype=np.int32) + -1
same_image_questions_neg = np.zeros((len(question_ids), 300), dtype=np.int32) + -1

for idx in tqdm(range(len(question_negs))):
    neg_ids = question_values[idx]["top_k_questions_neg_trainval"]
    same_image_ids = question_values[idx]["same_image_questions_neg"]
    question_negs[idx][:len(neg_ids)] = neg_ids
    try:
        same_image_questions_neg[idx][:len(same_image_ids)] = same_image_ids
    except:
        import pdb
        pdb.set_trace()

In [46]:
cPickle.dump({"qids": question_ids, 
                    "question_negs": question_negs, 
                    "same_image_questions_neg": same_image_questions_neg}, 
             open(save_negs_paths[2], "wb"))
print(save_negs_paths[2])

../../datasets/VQA/back-translate/fil_trainval_question_negs.pkl


In [47]:
d = cPickle.load(open(save_negs_paths[2], "rb"))

In [53]:
d["question_negs"][0]

array([  116223003, -2041389054,     1558004, -1110880256,   572474001,
        1199842305,  1199842307, -1110880255, -1110880254,   174351001,
        -664583168,  -664583167,     1786001, -1615059968, -1615059967,
        -664583166,  -664583165, -1615059966, -1993780221,   468736003,
         468736002,   468736001,   468736000, -1993780222, -1993780223,
         287870022, -1942778877, -1942778878, -1993780224, -1942778879,
          93683005, -1483171838, -1483171839, -1483171840,    93666006,
         466627003,   228419001,  1099678721,  1099678723, -1036870653,
       -1036870654, -1036870655, -1036870656,    62543007,    25230001,
       -1193237504, -1193237503, -1193237502,   364748001,  1902339072,
        1902339073,  1902339074, -1193237501,   341173251,   341173250,
         341173249,   341173248,    91408001, -1305934847,  -546518013,
        -546518015,    89829003,    10376195,    10376194,    10376193,
          10376192,    86267005,   437080003,  1147841536,   654